# Detection Service

In [1]:
import os
import sys
import time
import io
import threading
from datetime import datetime
import random
sys.path.insert(0, os.path.abspath('..'))

import cv2
import requests
from  matplotlib import pyplot as plt
from IPython.display import clear_output
from ipywidgets import widgets
from PIL import Image, ImageDraw
from IPython.core.display import display
import numpy as np
import scipy.spatial

DETECTION_URL = 'http://boat-detect:8080/v1/detect'
STREAM_URL = 'http://10.10.117.249:8080/'

In [2]:
def feed_widget(livefd):
    timestamp, img, meta = livefd.snap()
    a = widgets.Image(value=pil2pngs(img),format='png')
    a.stop = False
    
    def loop():
        while not a.stop:
            time.sleep(0.5)
            timestamp, img, meta = livefd.snap()
            a.value = pil2pngs(img)
        
    threading.Thread(target=loop).start()
    return a

In [3]:
def pil2pngs(img):
    output = io.BytesIO()
    img.save(output, format='PNG')
    output.seek(0)
    img_data = output.read()
    output.close()
    
    return img_data

In [4]:
class RemoteCameraSource(object):
    
    def __init__(self, url):
        self._url = url
        self._thread = None
        self._stream = None
        self._connecting = False
        self._last_frame = None
        self._last_frame_time = None
        self._next = None
        
    def __del__(self):
        self.stop_stream()
        
    def snap(self):
        # returns (timestamp, image, metadata)
        if self._last_frame is not None:
            return (
                self._last_frame_time,
                Image.fromarray(
                    cv2.cvtColor(self._last_frame, cv2.COLOR_BGR2RGB), 
                    "RGB"),
                {}
            )
        else:
            return (None, None, {})
    
    def reconnect(self):
        while self._connecting:
            time.sleep(1)
                
        self._connecting = True
        
        connected = False
        while not connected:
            self._stream = cv2.VideoCapture(self._url)
            connected = self._stream.isOpened()
            if connected:
                self._connecting = False
                return
            
            print('Could not connect to {}'.format(self._url))
            print('Retrying in 1 minute')
            
            time.sleep(60)
            
    def start_stream(self):
        if self._thread and self._thread.is_alive():
            print('Thread already running')
            return
        
        self.reconnect()
        
        self._thread = threading.Thread(target=self._watch, args=())
        self._thread.start()
        
    def stop_stream(self):
        self._stream.release()
        
    def _watch(self):
        fail_counter = 0
        while self._stream.isOpened():
            frame_time = datetime.now()
            ret, frame = self._stream.read()
            
            if not ret:
                fail_counter += 1
            else:
                fail_counter = 0
                self._last_frame_time = frame_time
                self._last_frame = frame
                if self._next:
                    self._run_next()
            
            time.sleep(0.01)
                
            if fail_counter > 50:
                self.reconnect()
                fail_counter = 0        
                
        self._stream.release()
        
    def _run_next(self):
        threading.Thread(target=self._next).start()
        
    def register(self, next_step):
        self._next = next_step

In [5]:
live_feed = RemoteCameraSource(STREAM_URL)
live_feed.start_stream()

In [6]:
while not live_feed.snap()[0]:
    time.sleep(1)
    
preview = feed_widget(live_feed)
preview    

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x05\x00\x00\x00\x02\xd0\x08\x02\x00\x00\x00@\x1fJ\x0…

In [7]:
class DetectionFilter(object):
    def __init__(self, source, url, overlay=False):
        self._source = source
        self._data = {}
        self._overlay = overlay
        self._url = url
        self._last_snap = self._source.snap()
        self._running = False
        
        self._source.register(self.detect)
        
    @property
    def overlay(self):
        return self._overlay
    
    @overlay.setter
    def overlay(self, val):
        self._overlay = bool(val)
        
    def snap(self):
        return self._last_snap
    
    def detect(self):
        if not self._running:
            self._running = True
            timestamp, img, metadata = self._source.snap()
            response = requests.post(
                self._url, files={"image": pil2pngs(img)}).json()

            if self._overlay:
                canvas = ImageDraw.Draw(img)
                for item in response['objects']:
                    canvas.rectangle(item['box'], outline='red')

            metadata.setdefault('detect', []).append(response)
            
            self._last_snap = (timestamp, img, metadata)
            self._running = False

In [8]:
pipeline = DetectionFilter(live_feed, DETECTION_URL, True)

while not pipeline.snap()[0]:
    time.sleep(1)
    
preview2 = feed_widget(pipeline)
preview2    

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x05\x00\x00\x00\x02\xd0\x08\x02\x00\x00\x00@\x1fJ\x0…

# Object Identification

In [9]:
#time0 = pipeline.snap()
time0 = (datetime(2021, 4, 22, 3, 40, 31, 994728),
 None,
 {'detect': [{'objects': [{'box': [207.0, 509.0, 299.0, 532.0],
      'confidence': 0.8941678404808044,
      'class': 'boats'},
     {'box': [65.0, 326.0, 136.0, 403.0],
      'confidence': 0.8731332421302795,
      'class': 'boats'},
     {'box': [1066.0, 109.0, 1142.0, 171.0],
      'confidence': 0.8629531264305115,
      'class': 'boats'},
     {'box': [82.0, 602.0, 149.0, 671.0],
      'confidence': 0.8608279228210449,
      'class': 'boats'},
     {'box': [298.0, 438.0, 383.0, 461.0],
      'confidence': 0.8419623374938965,
      'class': 'boats'},
     {'box': [242.0, 0.0, 298.0, 19.0],
      'confidence': 0.6364468932151794,
      'class': 'boats'}]}]})
time.sleep(1)
#time1 = pipeline.snap()
time1 = (datetime(2021, 4, 22, 3, 40, 32, 757943),
 None,
 {'detect': [{'objects': [{'box': [211.0, 508.0, 307.0, 532.0],
      'confidence': 0.8908742070198059,
      'class': 'boats'},
     {'box': [1073.0, 113.0, 1147.0, 176.0],
      'confidence': 0.8825204968452454,
      'class': 'boats'},
     {'box': [67.0, 325.0, 136.0, 405.0],
      'confidence': 0.8686448931694031,
      'class': 'boats'},
     {'box': [81.0, 601.0, 151.0, 672.0],
      'confidence': 0.8586115837097168,
      'class': 'boats'},
     {'box': [305.0, 438.0, 390.0, 462.0],
      'confidence': 0.8368647694587708,
      'class': 'boats'}]}]})

In [10]:
# calculate centroids of all objects
centroids0 = [np.mean(np.array(item['box']).reshape((2, 2)), axis=0)
              for item in time0[2]['detect'][-1]['objects']]
ids0 = np.array([random.randint(0,50) for i in range(len(centroids0))]) # make some fake ids like mclovin
centroids1 = [np.mean(np.array(item['box']).reshape((2, 2)), axis=0)
              for item in time1[2]['detect'][-1]['objects']]

# calcualte distance between all new and all old objects
distances = scipy.spatial.distance.cdist(centroids1, centroids0)

# assume the shortest distance moved is the match
new2old = [distances[idx].argmin() for idx in range(distances.shape[0])]
ids1 = ids0[new2old]

print(ids0)
print(ids1)

[33 20 47  9 33 12]
[33 47 20  9 33]
